In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# === CONFIG ===
FOLDER = "randomSearchv3" 
# INPUT_CSV = os.path.join("data", FOLDER, "csv", "detailed_summary_report_10.csv")
# BASE_OUTPUT_DIR = os.path.join("data", FOLDER, "charts_10")

# INPUT_CSV = os.path.join("data", FOLDER, "combine", "MEAN-combine_summary_report.csv")
INPUT_CSV = os.path.join("data", FOLDER, "combine", "FIRST-combine_summary_report.csv")
BASE_OUTPUT_DIR = os.path.join("data", FOLDER, "combine-FIRST")

ALPHA_PLOT_DIR = os.path.join(BASE_OUTPUT_DIR, "alpha_analysis")
COMPARISON_DIR = os.path.join(BASE_OUTPUT_DIR, "compare_best_alpha")
os.makedirs(ALPHA_PLOT_DIR, exist_ok=True)
os.makedirs(COMPARISON_DIR, exist_ok=True)

# === LOAD DATA ===
df = pd.read_csv(INPUT_CSV)
df["LevyFlight.alpha"] = pd.to_numeric(df["LevyFlight.alpha"], errors="coerce")

# Fill missing coverage with 0 for consistency
for col in ["avg_coverage", "min_coverage", "max_coverage"]:
    df[col] = df[col].fillna(0)

# === STEP 1: Analyze alpha for LevyFlight ===
levy_df = df[df["Agent_movementModel"] == "LevyFlight"].copy()
alpha_group = levy_df.groupby("LevyFlight.alpha")["avg_coverage"].mean()
best_alpha = alpha_group.idxmax()
print(f"\n✅ Best alpha for LevyFlight: {best_alpha}")

# === STEP 2: Plot all alpha values
for alpha in sorted(levy_df["LevyFlight.alpha"].dropna().unique()):
    subset = levy_df[levy_df["LevyFlight.alpha"] == alpha]
    df_melt = subset.melt(
        id_vars=["Target_movementModel"],
        value_vars=["min_coverage", "avg_coverage", "max_coverage"],
        var_name="Coverage Type",
        value_name="Coverage (%)"
    )
    plt.figure(figsize=(12, 6))
    sns.barplot(
        data=df_melt,
        x="Target_movementModel",
        y="Coverage (%)",
        hue="Coverage Type",
        palette="Set2"
    )
    plt.title(f"Coverage per Target (\u03b1 = {alpha})")
    plt.xticks(rotation=45)
    plt.tight_layout()
    out_path = os.path.join(ALPHA_PLOT_DIR, f"alpha_{alpha:.2f}_coverage.png")
    plt.savefig(out_path)
    plt.close()
    print(f"✅ Saved: {out_path}")

# === STEP 3: Filter data to only keep best alpha LevyFlight for comparisons ===
df_filtered = df.copy()
df_filtered = df_filtered[
    (df_filtered["Agent_movementModel"] != "LevyFlight") |
    ((df_filtered["Agent_movementModel"] == "LevyFlight") & (df_filtered["LevyFlight.alpha"] == best_alpha))
]

# === PLOT SETTINGS ===
sns.set(style="whitegrid")
palette = "Set2"

# === Plot 1: Min / Avg / Max Coverage % (split by target)
def plot_coverage_stats(df):
    df_melt = df.melt(
        id_vars=["Agent_movementModel", "Target_movementModel"],
        value_vars=["min_coverage", "avg_coverage", "max_coverage"],
        var_name="Coverage Type",
        value_name="Coverage (%)"
    )
    for target_model, group in df_melt.groupby("Target_movementModel"):
        output_dir = os.path.join(COMPARISON_DIR, str(target_model))
        os.makedirs(output_dir, exist_ok=True)

        plt.figure(figsize=(12, 6))
        sns.pointplot(
            data=group,
            x="Agent_movementModel",
            y="Coverage (%)",
            hue="Coverage Type",
            dodge=True,
            palette=palette
        )
        plt.title(f"Min, Avg, Max Coverage % (Target: {target_model}, Best α = {best_alpha})")
        plt.xticks(rotation=45)
        plt.tight_layout()
        out_path = os.path.join(output_dir, f"coverage_min_avg_max_{target_model}.png")
        plt.savefig(out_path)
        plt.close()
        print(f"✅ Saved: {out_path}")

# === Plot 2–5: Time to reach coverage threshold

def plot_time_to_coverage(df, threshold):
    column = f"time_to_{threshold}_percent"
    df_plot = df[["Agent_movementModel", "Target_movementModel", column]].dropna()
    for target_model, group in df_plot.groupby("Target_movementModel"):
        output_dir = os.path.join(COMPARISON_DIR, str(target_model))
        os.makedirs(output_dir, exist_ok=True)

        plt.figure(figsize=(12, 6))
        sns.barplot(
            data=group,
            x="Agent_movementModel",
            y=column,
            palette=palette
        )
        plt.title(f"Time to {threshold}% Coverage — Target: {target_model} (Best α = {best_alpha})")
        plt.ylabel("Time (seconds)")
        plt.xticks(rotation=45)
        plt.tight_layout()
        out_path = os.path.join(output_dir, f"time_to_{threshold}_coverage_{target_model}.png")
        plt.savefig(out_path)
        plt.close()
        print(f"✅ Saved: {out_path}")

# === GENERATE FINAL CHARTS ===
plot_coverage_stats(df_filtered)
for pct in [25, 50, 75, 90]:
    plot_time_to_coverage(df_filtered, pct)

print(f"\n🎉 All plots and alpha analysis saved under: {BASE_OUTPUT_DIR}")



✅ Best alpha for LevyFlight: 0.25
✅ Saved: data\randomSearchv3\combine-FIRST\alpha_analysis\alpha_0.25_coverage.png
✅ Saved: data\randomSearchv3\combine-FIRST\alpha_analysis\alpha_0.50_coverage.png
✅ Saved: data\randomSearchv3\combine-FIRST\alpha_analysis\alpha_0.75_coverage.png
✅ Saved: data\randomSearchv3\combine-FIRST\alpha_analysis\alpha_1.00_coverage.png
✅ Saved: data\randomSearchv3\combine-FIRST\alpha_analysis\alpha_1.25_coverage.png
✅ Saved: data\randomSearchv3\combine-FIRST\alpha_analysis\alpha_1.50_coverage.png
✅ Saved: data\randomSearchv3\combine-FIRST\alpha_analysis\alpha_1.75_coverage.png
✅ Saved: data\randomSearchv3\combine-FIRST\alpha_analysis\alpha_2.00_coverage.png
✅ Saved: data\randomSearchv3\combine-FIRST\compare_best_alpha\StationaryClustered\coverage_min_avg_max_StationaryClustered.png
✅ Saved: data\randomSearchv3\combine-FIRST\compare_best_alpha\StationaryClustered\time_to_25_coverage_StationaryClustered.png


C:\Users\ASUS\AppData\Local\Temp\ipykernel_7520\3215538316.py:109: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
C:\Users\ASUS\AppData\Local\Temp\ipykernel_7520\3215538316.py:109: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


✅ Saved: data\randomSearchv3\combine-FIRST\compare_best_alpha\StationaryClustered\time_to_50_coverage_StationaryClustered.png
✅ Saved: data\randomSearchv3\combine-FIRST\compare_best_alpha\StationaryClustered\time_to_75_coverage_StationaryClustered.png


C:\Users\ASUS\AppData\Local\Temp\ipykernel_7520\3215538316.py:109: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
C:\Users\ASUS\AppData\Local\Temp\ipykernel_7520\3215538316.py:109: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


✅ Saved: data\randomSearchv3\combine-FIRST\compare_best_alpha\StationaryClustered\time_to_90_coverage_StationaryClustered.png

🎉 All plots and alpha analysis saved under: data\randomSearchv3\combine-FIRST
